## [ 한국은행ECOS API 활용 ] 소비자 동향 조사 - 서울 제외 지역별 현재가계부채CSI 정보 수집


### 0. 과제 개요

In [ ]:
'''

[과제 개요]

1. 과제 목표 : 소비자 심리지수(소비자들의 경제에 대한 인식, 전망, 소비 심리를 지수화) 중 현재 가계부채 항목의 데이터를 수집해 소비 심리 변화 관찰

2. 수집 대상 : 2024년1월부터 2025년1월까지의 소비자 동향 조사 중 지역별 현재가계부채CSI 데이터

3. 데이터 출처 : 한국은행 Open API (https://ecos.bok.or.kr/api/#/)

4. API 활용 방법 : 한국은행 Open API 서비스의 개발 가이드를 참고하여 인증키, 통계표 코드 등을 조합해 호출

'''

###1. 환경 설정 및 API URL 구성

##### VBA 코드 : [511Y004] / CSI 코드 : [FMDB] - 현재가계부채


In [ ]:
pip install requests bs4 lxml

In [ ]:
import requests, bs4
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 상세주소구성 : (https://ecos.bok.or.kr/api) / 서비스명 / 인증키 / 요청유형 / 언어구분 / 요청시작건수 / 요청종료건수 / 통계표코드 / 주기 / 검색시작일자 / 검색종료일자 / 통계항목코드1 / 통계항목코드2

# 구성부
authKey = "" # 인증키
requestType = "xml"
lang = "kr"
startReqCase = "1"
endReqCase = "10" # 전부 다 불러오려면 xml의 totalCount값 가져와서 넣기
tableCode = "511Y004"
cycle = "M"
startDate = "202401"
endDate = "202503"
itemCode1 = "FMDB"

# API URL
apiURL = f"https://ecos.bok.or.kr/api/StatisticSearch/{authKey}/{requestType}/{lang}/{startReqCase}/{endReqCase}/{tableCode}/{cycle}/{startDate}/{endDate}/{itemCode1}"

# XML 데이터 1개만 조회해서 totalCount 값 확인하는 용도의 url
getTotalCntURL = f"https://ecos.bok.or.kr/api/StatisticSearch/{authKey}/{requestType}/{lang}/1/1/{tableCode}/{cycle}/{startDate}/{endDate}/{itemCode1}"


### 2. XML 문서를 파싱

#### 2-1. 데이터의 총 건수를 가져와서 저장

In [ ]:
# 데이터의 총 개수 파악을 위해 getTotalCntUrl로 데이터의 건수를 먼저 받아옴
resp = requests.get(getTotalCntURL)
resp.encoding = "utf-8"
html = resp.text
bs = bs4.BeautifulSoup(html, features="xml")

# list_total_count의 총 건수를 int로 변환 후 저장
totalCnt = int(bs.find(name = "list_total_count").text)
totalCnt

194

#### 2-2. 모든 건수를 반환하는 api URL로 다시 XML 파싱

In [ ]:
# 완성된 URL : 요청종료건수(endReqCase)를 받아온 총 건수(totalCnt)로 변경
allCaseURL = f"https://ecos.bok.or.kr/api/StatisticSearch/{authKey}/{requestType}/{lang}/{startReqCase}/{totalCnt}/{tableCode}/{cycle}/{startDate}/{endDate}/{itemCode1}"

# 새 URL로 다시 파싱
resp = requests.get(allCaseURL)
resp.encoding = "utf-8"
html = resp.text
allCaseXML = bs4.BeautifulSoup(html, features="xml")
allCaseXML

<?xml version="1.0" encoding="utf-8"?>
<StatisticSearch>
<list_total_count number="true">194</list_total_count>
<row>
<STAT_CODE>511Y004</STAT_CODE>
<STAT_NAME>6.2.4. 소비자동향조사(지역, 월)</STAT_NAME>
<ITEM_CODE1>FMDB</ITEM_CODE1>
<ITEM_NAME1>현재가계부채CSI</ITEM_NAME1>
<ITEM_CODE2>Z11</ITEM_CODE2>
<ITEM_NAME2>부산</ITEM_NAME2>
<ITEM_CODE3/>
<ITEM_NAME3/>
<ITEM_CODE4/>
<ITEM_NAME4/>
<UNIT_NAME/>
<WGT/>
<TIME>202401</TIME>
<DATA_VALUE>102</DATA_VALUE>
</row>
<row>
<STAT_CODE>511Y004</STAT_CODE>
<STAT_NAME>6.2.4. 소비자동향조사(지역, 월)</STAT_NAME>
<ITEM_CODE1>FMDB</ITEM_CODE1>
<ITEM_NAME1>현재가계부채CSI</ITEM_NAME1>
<ITEM_CODE2>Z12</ITEM_CODE2>
<ITEM_NAME2>대구경북</ITEM_NAME2>
<ITEM_CODE3/>
<ITEM_NAME3/>
<ITEM_CODE4/>
<ITEM_NAME4/>
<UNIT_NAME/>
<WGT/>
<TIME>202401</TIME>
<DATA_VALUE>100</DATA_VALUE>
</row>
<row>
<STAT_CODE>511Y004</STAT_CODE>
<STAT_NAME>6.2.4. 소비자동향조사(지역, 월)</STAT_NAME>
<ITEM_CODE1>FMDB</ITEM_CODE1>
<ITEM_NAME1>현재가계부채CSI</ITEM_NAME1>
<ITEM_CODE2>Z14</ITEM_CODE2>
<ITEM_NAME2>광주전남</ITEM_NAME2>
<ITEM_CO

### 3. XML 태그에서 원하는 정보 추출해서 리스트로 저장

In [ ]:
# 데이터 프레임에 표시할 제목, 지역, 연/월 리스트 선언
title = ""
regionList = []
dateList = []
csiList = []

# 모든 row값 저장
rows = allCaseXML.find_all("row")
try:
    for i in range(len(rows)):
        region = rows[i].find("ITEM_NAME2").text
        date = rows[i].find("TIME").text
        value = rows[i].find("DATA_VALUE").text

        # 날짜 형식 변환 ( YYYY년/ MM월 )
        changeDate = f"{date[:4]}년 {date[4:]}월"

        regionList.append(region)
        dateList.append(changeDate)
        csiList.append(value)
except Exception as e :
    print(f"데이터 수집 중 에러 발생 : {e}")

#### 3-1. 데이터프레임으로 만들기

In [ ]:
csiDf = pd.DataFrame({
    "지역": regionList,
    "연/월": dateList,
    "CSI 지수": csiList
})
csiDf

,지역,연/월,CSI 지수
0,부산,2024년 01월,102
1,대구경북,2024년 01월,100
2,광주전남,2024년 01월,103
3,전북,2024년 01월,102
4,대전세종충남,2024년 01월,99
...,...,...,...
189,인천,2025년 03월,99
190,제주,2025년 03월,104
191,경기,2025년 03월,96
192,강릉,2025년 03월,99


### 3-2. 피벗테이블 변환 ( 지역과 연월 컬럼의 중복을 줄여보고 싶어서 웹 사이트를 참고했고 수집한 데이터의 의미를 알아보고자 GPT를 활용했습니다. )

In [ ]:
csiTable = csiDf.pivot(index="지역", columns="연/월", values="CSI 지수")
csiTable

연/월,2024년 01월,2024년 02월,2024년 03월,2024년 04월,2024년 05월,2024년 06월,2024년 07월,2024년 08월,2024년 09월,2024년 10월,2024년 11월,2024년 12월,2025년 01월,2025년 02월,2025년 03월
지역,,,,,,,,,,,,,,,
강릉,101,101,105,99,99,98,99,98,100,97,97,97,100,95,99
강원,101,102,105,101,102,101,103,100,101,100,99,100,102,100,101
경기,98,100,99,99,99,98,99,100,97,100,99,98,99,97,96
경남,101,102,103,99,98,99,102,100,100,102,101,101,100,101,NaN
광주전남,103,104,104,106,107,106,104,105,104,104,104,105,102,105,103
대구경북,100,97,96,98,100,97,97,99,100,97,98,97,98,98,96
대전세종충남,99,100,101,101,100,102,101,101,104,102,101,103,103,101,102
부산,102,103,101,98,100,98,101,100,101,102,102,102,102,99,101
울산,99,99,98,98,99,99,99,99,101,100,100,99,99,98,102


In [ ]:
'''

[ GPT가 분석한 결과 정리 ]

1. 소비자 심리 지수의 판단 기준
    - 100 초과면 "경기 낙관"
    - 100 미만이면 "경기 비관" 으로 해석

2. 전반적인 흐름 : CSI 97~103 중심의 완만한 움직임, 대체로 안정적

3. 지역별 포인트
    - 광주/전남 : 전국에서 가장 낙관적인 소비심리, 지속적으로 높음
    - 경기 : 수도권임에도 CSI가 낮은 편, 구조적 요인 가능성 (금리, 부동산 등)
    - 대구경북 : 일관된 낮은 CSI, 경기 비관적 심리 지속
    - 계절성 : 연말→연초 하락세, 연말 소비 특수 후 감소, 설 연휴 전후 지출 억제 경향 등 계절적 요인으로 해석 가능

'''